In [7]:
import re
import os
import json
import statistics
import datetime
import warnings
from src.najdi_rok import najdi_rok
import polars as pl
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [8]:
df = pl.read_parquet(os.path.join("data","cnb_vyber.parquet"))
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))

In [9]:
preklady = df.explode("041_a").drop_nulls(subset=["041_a","rok"]).filter(pl.col("041_a") != "cze")

In [10]:
preklady_celkem = preklady.group_by(['rok']).len().rename({'len':'prekladu_celkem'})
preklady_celkem

rok,prekladu_celkem
i64,u32
1971,363
1986,298
1974,390
1968,372
1983,349
…,…
1944,94
1953,121
1956,205


In [11]:
nahradit = {
    'eng': 'angličtina', 'ger': 'němčina', 'slo': 'slovenština', 'rus': 'ruština', 'fre': 'francouzština', 'lat': 'latina'
}

In [12]:
nejzastoupenejsi_jazyky = preklady.group_by('041_a').len().sort('len', descending=True)
nejzastoupenejsi_jazyky = nejzastoupenejsi_jazyky.get_column('041_a').to_list()
nejzastoupenejsi_jazyky = nejzastoupenejsi_jazyky[0:6]
print(nejzastoupenejsi_jazyky)
preklady_po_jazycich = preklady.filter(pl.col("rok").is_between(1900,2020)).group_by(['rok','041_a']).len().rename({'len':'prekladu'})
preklady_po_jazycich = preklady_po_jazycich.join(preklady_celkem, on='rok', how='full')
preklady_po_jazycich = preklady_po_jazycich.with_columns((pl.col('prekladu') / pl.col('prekladu_celkem')).alias('podíl'))
preklady_po_jazycich = preklady_po_jazycich.with_columns(pl.col('041_a').cast(str))
preklady_po_jazycich = preklady_po_jazycich.filter(pl.col("041_a").is_in(nejzastoupenejsi_jazyky)).sort('rok').rename({'041_a':'jazyk'}).with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
preklady_po_jazycich = preklady_po_jazycich.with_columns(pl.col('jazyk').replace_strict(nahradit))
preklady_po_jazycich

['eng', 'ger', 'slo', 'rus', 'fre', 'lat']


rok,jazyk,prekladu,rok_right,prekladu_celkem,podíl
datetime[μs],str,u32,i64,u32,f64
1900-01-01 00:00:00,"""francouzština""",5,1900,63,0.079365
1900-01-01 00:00:00,"""němčina""",20,1900,63,0.31746
1900-01-01 00:00:00,"""slovenština""",1,1900,63,0.015873
1900-01-01 00:00:00,"""ruština""",4,1900,63,0.063492
1900-01-01 00:00:00,"""latina""",21,1900,63,0.333333
…,…,…,…,…,…
2020-01-01 00:00:00,"""francouzština""",18,2020,833,0.021609
2020-01-01 00:00:00,"""slovenština""",145,2020,833,0.17407
2020-01-01 00:00:00,"""angličtina""",353,2020,833,0.42377


In [13]:
def irozhlas_conf():

    return {
        "config" : {
             "title": {'font': "Noticia Text",
                      'fontSize': 14,
                      'anchor': 'start',
                       'fontWeight': 'bolder',
                    'subtitleFont': 'Noticia Text',
                      'subtitleFontSize': 12,
                       'subtitleFontWeight': 'lighter',
                      'subtitlePadding': 12,
                       'dy': -12,
                      },
             "axis": {
                "labelFont": "Asap",
                "titleFont": "Asap",
                 "fontWeight": "lighter",
                "titleFontWeight": "lighter",
                "labelFontSize": 10,
                "titleFontSize": 10,
                'labelPadding': 2,
                 'titlePadding': 10
            },
            "legend": {
                "labelFont": "Asap",
                "labelFontWeight": "normal",
                "titleFont": "Asap",
                "titleFontWeight": "normal",
                "labelFontSize": 10,
                "titleFontSize": 10
            }
        }
    }

alt.themes.register('irozhlas', irozhlas_conf)
alt.themes.enable('irozhlas')

ThemeRegistry.enable('irozhlas')

In [14]:
titulek = "Z čeho se kdy překládalo do češtiny"
podtitulek = ["Podíl jazyků na překladové literatuře – krásné, naučné i odborné."] #,"Angličtina a francouzština (téměř) zmizely jen během 2. světové války,","kdy dominovaly překlady z němčiny. Po osvobození raketově narostl podíl ruštiny."]
kredit = "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025"

In [15]:
podily = alt.Chart(preklady_po_jazycich.to_pandas(), title={'text': titulek, 'subtitle': podtitulek}).mark_area().encode(
    alt.X("rok:T", title=None),
    alt.Y('podíl:Q', axis=alt.Axis(labelExpr="datum.label * 100 + ' %'", orient='left'), title=None),
    alt.Color("jazyk:N", 
              scale=alt.Scale(range=['#a7ddd6','#001f80','#e5db00','#e49996','#9cc002']), 
              title=None, legend=alt.Legend(direction='horizontal',orient='none',  legendY=-30)),
    alt.Order('jazyk:N', sort='ascending')
).properties(width=400, height=200)

credits = alt.Chart({"values": [{"text": kredit}]}) \
.mark_text(
    dx=33,  # Adjust horizontal position (negative moves left)
    dy=140,  # Adjust vertical position (negative moves up)
    fontSize=11,
    font='Asap',
    color='#222'
).encode(
    text='text:N',
#    x=alt.value(400),  # Position from right edge
)

final_chart = (podily + credits)

final_chart.configure_axis(
    grid=False
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [31]:
podily_faceted = alt.Chart(preklady_po_jazycich.to_pandas(), title={'text': titulek, 'subtitle': podtitulek}, width=300).mark_area().encode(
    alt.X("rok:T", 
          title=None,
          axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
    ),
    alt.Y('podíl:Q', 
          axis=alt.Axis(labelExpr="datum.label * 100 + ' %'", orient='right', domainOpacity=0, tickColor='#DCDDD6'), 
          title=None
    ),
    alt.Color("jazyk:N", 
              scale=alt.Scale(range=['#687fa9','#84c0e4','#E0DAB5','#e3d83b','#b45058','#789256']), ## '# ['#a7ddd6','#001f80','#e6e6e4','#e5db00','#e49996','#9cc002']), 
              title=None, 
              legend=None
    ),
    row=alt.Row(
        "jazyk:N", 
        title=None, 
        spacing=15, 
        header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='middle', labelFont='Asap')
    )
).resolve_axis(
    x='independent'
).properties(height=60, width=300)

podily_faceted

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [17]:
def me_to_neurazi(zaklad_grafu, soubor, kredity):

    os.makedirs("grafy", exist_ok=True)
    
    credits = pl.DataFrame({'text': [kredity]})
    credits = alt.Chart(credits.to_pandas(), width=300).encode(text=alt.Text('text:N')).mark_text(fontSize=10,font="Asap", baseline='line-bottom')
    combined_chart = alt.vconcat(zaklad_grafu, credits)

    combined_chart.save(os.path.join("grafy",f"{soubor}.svg"))
    combined_chart.save(os.path.join("grafy",f"{soubor}.png"), ppi=400)
    
    return combined_chart

In [18]:
me_to_neurazi(podily_faceted, "zdrojove_jazyky", "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.

alt.VConcatChart(...)